<a href="https://colab.research.google.com/github/Manggy94/PokerBrain/blob/master/Hands_Recorder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

This notebook is intended to record Hands from .txt files of different poker rooms, especially WINAMAX. To do so we'll use special objects from custom python modules: FileParser and HandConverter. Doc can be found on GitHub: https://github.com/Manggy94/PokerBrain.

#Preparation steps

First, We have to mount drive and verify its content, to add it to our path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Then we can import necessary libraries and modules



In [ ]:
!pip install treys
import sys
import os
sys.path.append('/content/drive/MyDrive/PokerBrain/PokerBrain')
from preprocessor import Preprocessor
import numpy as np
import pandas as pd
import datetime
import multiprocessing

In [ ]:
os.getcwd()

In [ ]:
if os.getcwd() != "/content/drive/MyDrive/PokerBrain/PokerBrain":
  os.chdir(f"{os.getcwd()}/drive/MyDrive/PokerBrain/PokerBrain")

In [ ]:
pp = Preprocessor()

In [ ]:
def register_hands(parse_all: bool=False):
  files = pp.conv.parser.get_holdem_game_files("history")
  if parse_all:
    stored_files = np.array([])
  else:
    stored_files = np.array([filename.replace(".csv","") for filename in os.listdir(f"Histories/Hands")])
  pending = np.array([x for x in files if x.replace(".txt", "") not in stored_files])
  print(pending.size)
  count_files, count_stored = files.size, stored_files.size
  total = pending.size
  print(f"There are {total} Files to register")
  count=0
  actual_count = 0
  for file_ in pending:
    print(file_)
    count+=1
    np_hands = pp.conv.parser.parse_file(file_)
    print(np_hands)
    np_hands = pp.conv.filter(np_hands)
    if np_hands.size > 0:
      print(f"File n°{count}: {np_hands.size} hands")
      actual_count+=1
      hands = pp.conv.build_hands(np_hands)
      hands.to_csv(f"{os.getcwd()}/Histories/Hands/{file_.replace('.txt', '')}.csv")
    else:
      print(f"File n°{count}: Empty")
    print(f"({count}/{total}): {np.round(count/total*100, 2)}%")
  print(f"{actual_count}/{total} files have been registered with at least one hand")


In [ ]:
register_hands()

In [ ]:
stored_files = np.array([filename.replace(".csv","") for filename in os.listdir(f"Histories/Hands")])
data = pd.concat([pd.read_csv(f"{os.getcwd()}/Histories/Hands/{file}.csv", index_col=0) for file in stored_files], ignore_index=True)

In [ ]:
data.head()

In [ ]:
data.shape

(88185, 591)

In [ ]:
data.to_csv(f"{os.getcwd()}/Data/combo_data_tab.csv")

In [ ]:
interest_pos = ["UTG_", "HJ_", "CO_", "BTN_", "SB_", "BB_"]

In [ ]:
from API.listings import str_hands

In [ ]:
dico = {}
for x in str_hands:
  dico[x] = f"{x}"
dico["22.0"] = "22"
dico["33.0"] = "33"
dico["44.0"] = "44"
dico["55.0"] = "55"
dico["66.0"] = "66"
dico["77.0"] = "77"
dico["88.0"] = "88"
dico["99.0"] = "99"
dico["None"] = "None"
card_cols = [x for x in data.columns if "Card" in x]
hero_cols = [x for x in data.columns if "hero" in x]
flop_cols = [x for x in data.columns if "is" in x or "has" in x]

In [ ]:
to_concat =[]
for pos in interest_pos:
  n = len(pos)
  cols = [x for x in data.columns if pos in x ]
  new_cols = [x[n:] for x in cols]
  cols = ["level", "bb", "max_pl", "current_street"]+ hero_cols + cols + card_cols + flop_cols 
  new_cols = ["level", "bb", "max_pl", "current_street"] + hero_cols + new_cols + card_cols + flop_cols + ["position"]
  data_tab = data[cols].to_numpy()
  position = np.ones((data.shape[0],1)).astype(str)
  position[:] = pos[:-1]
  data_tab = np.concatenate((data_tab, position), axis=1)
  concat_df = pd.DataFrame(columns=new_cols, data=data_tab)
  to_concat.append(concat_df)
obs_data= pd.concat(to_concat).reset_index(drop=True)
obs_data["hand"] = obs_data["hand"].map(dico)
obs_data["hero_hand"] = obs_data["hero_hand"].map(dico)
#del(to_concat, n, new_cols, cols, position, concat_df, data_tab, dico)

In [ ]:
obs_data

In [ ]:
filter1 =~ obs_data["hand"].isin(["None", np.NaN])
shown_hands = obs_data[filter1].reset_index(drop=True)
filter2 = shown_hands["max_pl"] <7
short_handed = shown_hands[filter2].reset_index(drop=True)
filter3 = short_handed["current_street"] == "SD"
WTSD = short_handed[filter3].reset_index(drop=True)
filter4 = short_handed["hero_position"] != short_handed["position"]
training_hands = short_handed[filter4].reset_index(drop=True)

In [ ]:
features, hand_labels, combo_labels = shown_hands.drop(["hand", "combo"], axis=1), shown_hands["hand"], shown_hands["combo"]

In [ ]:
features.shape, hand_labels.shape, combo_labels.shape

((135710, 101), (135710,), (135710,))

In [ ]:
filter = features["hero_position"] != features["position"]
training_features = features[filter].reset_index(drop=True)
training_combos=combo_labels[filter].reset_index(drop=True)
training_hands=hand_labels[filter].reset_index(drop=True)

In [ ]:
features.to_csv(f"{os.getcwd()}/Data/features_tab.csv")
features

In [ ]:
training_features.to_csv(f"{os.getcwd()}/Data/training_features.csv")
training_features

In [ ]:
hand_labels.to_csv(f"{os.getcwd()}/Data/hand_labels_tab.csv")
hand_labels

In [ ]:
training_hands.to_csv(f"{os.getcwd()}/Data/training_hands.csv")
training_hands

In [ ]:
combo_labels.to_csv(f"{os.getcwd()}/Data/combo_labels_tab.csv")
combo_labels

In [ ]:
training_combos.to_csv(f"{os.getcwd()}/Data/training_combos.csv")
training_combos